In [3]:
!pip install trax

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 637.9/637.9 kB 11.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 21.12.2 requires cupy-cuda115, which is not installed.
tfx-bsl 1.12.0 requires google-api-python-client<2,>=1.7.11, but you have google-api-python-client 2.83.0 which is incompatible.
tfx-bsl 1.12.0 requires pyarrow<7,>=6, but you have pyarrow 5.0.0 which is incompatible.
tensorflow-transform 1.12.0 requires pyarrow<7,>=6, but you have pyarrow 5.0.0 which is incompatible.
onnx 1.13.1 requires protobuf<4,>

In [111]:
import trax
import trax.layers as tl
import trax.fastmath.numpy as np
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from string import punctuation
import random as rnd
from trax.supervised.training import TrainTask,EvalTask ,Loop

In [112]:
data=[]
with open('/kaggle/input/shakespeare-plays/alllines.txt') as lines:
    for line in lines :
        data.append(line.strip().replace('"',''))
        
    

In [113]:
def tokenize_data(data):
    clean_data=[]
    for line in data:
        line=word_tokenize(line)
        words=[]
        for word in line:
            if word not in punctuation:
                words.append(word.lower())
            
        clean_data.append(words)
    return(clean_data)

In [114]:
clean_data=tokenize_data(data)
clean_data[0:5]

[['act', 'i'],
 ['scene', 'i.', 'london', 'the', 'palace'],
 ['enter',
  'king',
  'henry',
  'lord',
  'john',
  'of',
  'lancaster',
  'the',
  'earl',
  'of',
  'westmoreland',
  'sir',
  'walter',
  'blunt',
  'and',
  'others'],
 ['so', 'shaken', 'as', 'we', 'are', 'so', 'wan', 'with', 'care'],
 ['find', 'we', 'a', 'time', 'for', 'frighted', 'peace', 'to', 'pant']]

In [117]:
def get_vocab(data):
    token_number={'<pad>':0,'<eos>':1}
    number_token={0:'<pad>',1:'<eos>'}
    for line in data:
        for word in line:
            if word not in token_number:
                token_number[word]=len(token_number)
                number_token[len(number_token)]=word
    return token_number,number_token
token_number,number_token=get_vocab(clean_data)
len(token_number),len(number_token)

(25986, 25986)

In [118]:
def get_tensor(data,vocab):
    tensor=[]
    for word in data :
        tensor.append(vocab[word])
    tensor.append(vocab['<eos>'])
    return(tensor)

In [122]:
def get_word(tensor,vocab):
    words=[]
    for number in tensor :
        words.append(vocab[number])
    return(words)

In [123]:
print(clean_data[2])
tensor=get_tensor(clean_data[2],token_number)
words=get_word(tensor,number_token)
print(tensor)
print(words)

['enter', 'king', 'henry', 'lord', 'john', 'of', 'lancaster', 'the', 'earl', 'of', 'westmoreland', 'sir', 'walter', 'blunt', 'and', 'others']
[9, 10, 11, 12, 13, 14, 15, 7, 16, 14, 17, 18, 19, 20, 21, 22, 1]
['enter', 'king', 'henry', 'lord', 'john', 'of', 'lancaster', 'the', 'earl', 'of', 'westmoreland', 'sir', 'walter', 'blunt', 'and', 'others', '<eos>']


In [58]:
def generator(data,vocab,batch_size=256,shuffle=True):
    data_size=len(data)
    index=[*range(data_size)]
    idx=0
    while True:
        if idx>=data_size:
            idx=0
        if shuffle:
            rnd.shuffle(index)
        tensor=np.array(get_tensor(data[index[idx]],vocab))
        idx+=1
        yield ((tensor,tensor))





In [59]:
X,Y=next(generator(clean_data,token_number))
X.shape

(10,)

In [60]:
X.shape,Y.shape

((10,), (10,))

In [61]:
train_size=int(len(clean_data)*.8)
val_size=int(len(clean_data)*.9)

train_data=clean_data[:train_size]
val_data=clean_data[train_size:val_size]
test_data=clean_data[val_size:]
print(len(train_data))
print(len(val_data))
print(len(test_data))

89116
11140
11140


In [62]:
boundaries =  [8,   16,  32, 64, 128, 256, 512]
batch_sizes = [256, 128, 64, 32, 16,    8,   4,  2]

# Create the generators.
train_batch_stream = trax.data.BucketByLength(
    boundaries, batch_sizes,
    length_keys=[0, 1]  # As before: count inputs and targets to length.
)(generator(train_data,token_number))

eval_batch_stream = trax.data.BucketByLength(
    boundaries, batch_sizes,
    length_keys=[0, 1]  # As before: count inputs and targets to length.
)(generator(val_data,token_number))
test_batch_stream = trax.data.BucketByLength(
    boundaries, batch_sizes,
    length_keys=[0, 1]  # As before: count inputs and targets to length.
)(generator(test_data,token_number))

In [64]:
x,y=next(train_batch_stream)
x.shape,y.shape

((128, 16), (128, 16))

In [65]:
train_generator=trax.data.AddLossWeights(id_to_mask=vocab['<pad>'])(train_batch_stream)
val_generator=trax.data.AddLossWeights(id_to_mask=vocab['<pad>'])(eval_batch_stream)
test_generator=trax.data.AddLossWeights(id_to_mask=vocab['<pad>'])(test_batch_stream)

In [66]:
x,y,m=next((train_generator))
x.shape,y.shape,m.shape

((256, 8), (256, 8), (256, 8))

In [76]:
def model(vocab, d_model=512, n_layers=2, mode='train'):
    vocab_size=len(vocab)
    model = tl.Serial( 
      tl.ShiftRight(mode='train'), # Stack the ShiftRight layer
      tl.Embedding(vocab_size,d_model), # Stack the embedding layer
      [tl.GRU(d_model) for _ in range(n_layers)], # Stack GRU layers of d_model units keeping n_layer parameter in mind (use list comprehension syntax)
      tl.Dense(vocab_size), # Dense layer
      tl.LogSoftmax(), # Log Softmax
    ) 
    ### END CODE HERE ###
    return model

In [68]:
classifier=model(256,vocab,1)

In [69]:
classifier

Serial[
  Serial[
    ShiftRight(1)
  ]
  Embedding_25986_256
  LSTM_256
  LSTM_512
  Dense_25986
  LogSoftmax
]

In [72]:
# PLEASE, DO NOT MODIFY OR DELETE THIS CELL
from trax.supervised import training
import itertools
def get_train_eval_tasks(train_generator,val_generator):
    
    rnd.seed(271)

    train_task = training.TrainTask(
        labeled_data=train_generator,
        loss_layer=tl.CrossEntropyLoss(),
        optimizer=trax.optimizers.Adam(0.0001),
        n_steps_per_checkpoint=10,
    )

    eval_task = training.EvalTask(
        labeled_data=val_generator,        
        metrics=[tl.CrossEntropyLoss(), tl.Accuracy()],
    )
    
    return train_task, eval_task
    

train_task, eval_task = get_train_eval_tasks(train_generator,val_generator)

In [110]:
len(train_data)/32

2784.875

In [73]:
dir_path = './model/'

n_step=10

def trainig (classifier,n_step,train_task,eval_task,dir_path):
    train=training.Loop(classifier,train_task,eval_tasks=[eval_task], output_dir=dir_path)
    train.run(n_step)
    return train

In [78]:
num_class=len(vocab)
n_dimension=128
classifier = model(vocab,n_dimension)
print('Show Sup_Layer in My Model\n\n',classifier)

Show Sup_Layer in My Model

 Serial[
  Serial[
    ShiftRight(1)
  ]
  Embedding_25986_128
  GRU_128
  GRU_128
  Dense_25986
  LogSoftmax
]


In [79]:
model=trainig (classifier,n_step,train_task,eval_task,'./model/')



Step      1: Total number of trainable weights: 6875778
Step      1: Ran 1 train steps in 28.63 secs
Step      1: train CrossEntropyLoss |  10.16531372
Step      1: eval  CrossEntropyLoss |  10.16522598
Step      1: eval          Accuracy |  0.10711953

Step     10: Ran 9 train steps in 102.71 secs
Step     10: train CrossEntropyLoss |  10.16471004
Step     10: eval  CrossEntropyLoss |  10.16403198
Step     10: eval          Accuracy |  0.09914795


In [84]:
x_test_,y_test,mask=next(test_generator)
x_test_.shape

(128, 16)

In [150]:
prediction=mo.eval_model(x_test_)
prediction.shape

(128, 16, 25986)

In [152]:
np.argmax(prediction,axis=-1).shape

(128, 16)

In [92]:
np.sum(np.argmax(prediction,axis=-1)==y_test)

DeviceArray(128, dtype=int32)

In [132]:
def generate_text(n_words,start_word,model,vocab):
    start_word=np.array([[vocab[start_word]]])
    prediction=model.eval_model(start_word)
    np.argmax(axis=-1)

(1, 2)

In [155]:
start_word=np.array([[vocab['play']]])
prediction=mo.eval_model(start_word)

In [142]:
np.argmax(prediction,axis=-1)

DeviceArray([[1]], dtype=int32)

In [156]:
prediction.[:,-1]

SyntaxError: invalid syntax (1607175486.py, line 1)

In [149]:
prediction.shape

(1, 1, 25986)

In [3]:
import os

def remove_folder_contents(folder):
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                remove_folder_contents(file_path)
                os.rmdir(file_path)
        except Exception as e:
            print(e)

folder_path = '/kaggle/working'
remove_folder_contents(folder_path)
os.rmdir(r'/kaggle/working/model')


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/model'

In [97]:
a=iter('hello')

In [105]:
vocab['play']

1823

In [107]:
inputs =  next(trax.data.tokenize(iter(['play']),
                                      keys=vocab))

TypeError: join() argument must be str or bytes, not 'NoneType'

In [163]:
a=np.array([1])

In [166]:
a[:,None].shape

(1, 1)

In [162]:
a[None,:]

DeviceArray([[[1, 2],
              [3, 4]]], dtype=int32)